## Summative Coursework - Experiments Investigating PRSH 

In [4]:
# Imports

import matplotlib.pyplot as plt
import numpy as np
import csv
import math
import random
import pandas as pd

from BSE import market_session

# graphing functions from BSE Intro Practical 


# Use this to plot trades of a single experiment
def plot_trades(trial_id):
    prices_fname = trial_id + '_transactions.csv'
    x = np.empty(0)
    y = np.empty(0)
    with open(prices_fname, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            time = float(row[1])
            price = float(row[2])
            x = np.append(x,time)
            y = np.append(y,price)

    plt.plot(x, y, 'x', color='black') 
    
# Use this to run an experiment n times and plot all trades
def n_runs_plot(n, trial_id, start_time, end_time, traders_spec, order_sched):
    x = np.empty(0)
    y = np.empty(0)

    for i in range(n):
        trialId = trial_id + '_' + str(i)
        tdump = open(trialId + '_avg_balance.csv','w')

        market_session(trialId, start_time, end_time, traders_spec, order_sched, tdump, True, False)
        
        tdump.close()

        with open(trialId + '_transactions.csv', newline='') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                time = float(row[1])
                price = float(row[2])
                x = np.append(x,time)
                y = np.append(y,price)

    plt.plot(x, y, 'x', color='black');

# !!! Don't use on it's own   
def getorderprice(i, sched, n, mode):
    pmin = min(sched[0][0], sched[0][1])
    pmax = max(sched[0][0], sched[0][1])
    prange = pmax - pmin
    stepsize = prange / (n - 1)
    halfstep = round(stepsize / 2.0)

    if mode == 'fixed':
        orderprice = pmin + int(i * stepsize)
    elif mode == 'jittered':
        orderprice = pmin + int(i * stepsize) + random.randint(-halfstep, halfstep)
    elif mode == 'random':
        if len(sched) > 1:
            # more than one schedule: choose one equiprobably
            s = random.randint(0, len(sched) - 1)
            pmin = min(sched[s][0], sched[s][1])
            pmax = max(sched[s][0], sched[s][1])
        orderprice = random.randint(pmin, pmax)
    return orderprice    

# !!! Don't use on it's own
def make_supply_demand_plot(bids, asks, title):
    # total volume up to current order
    volS = 0
    volB = 0

    fig, ax = plt.subplots()
    plt.ylabel('Price')
    plt.xlabel('Quantity')
    plt.title(title)
    pr = 0
    for b in bids:
        if pr != 0:
            # vertical line
            ax.plot([volB,volB], [pr,b], 'r-')
        # horizontal lines
        line, = ax.plot([volB,volB+1], [b,b], 'r-')
        volB += 1
        pr = b
    if bids:
        line.set_label('Demand')
        
    pr = 0
    for s in asks:
        if pr != 0:
            # vertical line
            ax.plot([volS,volS], [pr,s], 'b-')
        # horizontal lines
        line, = ax.plot([volS,volS+1], [s,s], 'b-')
        volS += 1
        pr = s
    if asks:
        line.set_label('Supply')
        
    if bids or asks:
        plt.legend()
    plt.show()

# Use this to plot supply and demand curves from supply and demand ranges and stepmode
def sup_dem(seller_num, sup_ranges, buyer_num, dem_ranges, stepmode, title):
    asks = []
    for s in range(seller_num):
        asks.append(getorderprice(s, sup_ranges, seller_num, stepmode))
    asks.sort()
    bids = []
    for b in range(buyer_num):
        bids.append(getorderprice(b, dem_ranges, buyer_num, stepmode))
    bids.sort()
    bids.reverse()
    
    make_supply_demand_plot(bids, asks, title) 

# plot sorted trades, useful is some situations - won't be used in this worksheet
def in_order_plot(trial_id):
    prices_fname = trial_id + '_transactions.csv'
    y = np.empty(0)
    with open(prices_fname, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            price = float(row[2])
            y = np.append(y,price)
    y = np.sort(y)
    x = list(range(len(y)))

    plt.plot(x, y, 'x', color='black')   

### Analysing How Changes In k Affect Behaviour
#### Market Conditions
I choose to emulate market conditions used by Vytelingum and Cliff/Snashall, inspired by Smith as these are well-documented experimental conditions believed to be varied enough to make strong claims about the ability of algorithms.

I also take inspiration from IBMs experiments, testing both balanced-group and one-to-many trader sets.

To change the k parameter either have to:
- Go into BSE manually for each run and alter k
- Alter BSE to be able to take parameters as input, perhaps through the market session?

TODO: write function in BSE that alters a global var that the PRSH initialiser takes value for k/mutate from 

In [ ]:
# Experimental setup
trials_per_market = 100 # important for valid results as PRSH is stochastic 

balanced_sellers_spec = [("ZIP", 5),("ZIC", 5),("PRSH", 5), ("SHVR", 5), ("GVWY", 5)]
balanced_buyers_spec = balanced_sellers_spec
balanced_traders_spec = {"sellers":balanced_sellers_spec, "buyers":balanced_buyers_spec}

start_time = 0
end_time = 180

In [ ]:
# CSV preparation

fname = 'balanced_market_output.csv'
tdump = open(fname, 'w')

# annotate columns
tdump.write('%s, %s,%s,%s, ' % ('expid', 'time','best bid','best offer')) 
for f in range(6):
        tdump.write('%s, %s, %s, %s, ' % ('type', 'balance', 'number of traders', 'profit per trader'))
tdump.write('%s, %s, ' % ('best bid','best offer'))
tdump.write('\n');


In [ ]:
## M1 experiment - symmetric supply/demand, range (10, 50), fixed stepmode, drip jitter order interval

sup_range = (10, 50)
dem_range = sup_range

supply_schedule = [{'from':start_time, 'to':end_time, "ranges":[sup_range], "stepmode":"fixed"}]
demand_schedule = supply_schedule

order_interval = 10
order_sched = {"sup":supply_schedule, "dem":demand_schedule, "interval":order_interval,"timemode":"drip-jitter"}

trial = 0
while trial < trials_per_market:
        # trial_id = 'trial%04d' % trial
        trial_id = trial
        market_session(trial_id, start_time, end_time, balanced_traders_spec, order_sched, tdump, False, True)
        tdump.flush()
        trial = trial + 1

### Analysing How Changing the Mutation Function Affects Behaviour For Set k

### Analysing an Adapted PRSH Trader

In [ ]:
# import improved PRSH from separate file
# analyse how changing parameters affect it in similar way to above?
# compare optimised parameter EPRSH and PRSH in each market to compare performance?